### Install SageMaker

In [2]:
!pip install -U sagemaker


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### 환경 구성

#### import module

In [3]:
import os
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.workflow.pipeline_context import PipelineSession


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


#### SageMaker Session Setup

In [4]:
sess = boto3.Session()
sm = sess.client("sagemaker")
role = get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

pipeline_session = PipelineSession()

model_package_group_name = "PipelineModelPackageGroup"
prefix = "pipeline-model-example"
pipeline_name = "serial-inference-pipeline"    # SageMaker Pipeline name

### Download California Housing dataset and upload to Amazon S3

In [5]:
# create data dirs
data_dir = os.path.join(os.getcwd(), "data")
os.makedirs(data_dir, exist_ok=True)

raw_dir = os.path.join(os.getcwd(), "data/raw")
os.makedirs(raw_dir, exist_ok=True)

In [6]:
# download data from s3
s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-example-files-prod-{region}",
    "datasets/tabular/california_housing/cal_housing.tgz",
    "cal_housing.tgz",
)

In [7]:
!tar -zxf cal_housing.tgz

tar: CaliforniaHousing/cal_housing.data: Cannot change ownership to uid 10017, gid 166: Operation not permitted
tar: CaliforniaHousing/cal_housing.domain: Cannot change ownership to uid 10017, gid 166: Operation not permitted
tar: Exiting with failure status due to previous errors


In [8]:
columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
    "medianHouseValue",
]
cal_housing_df = pd.read_csv("CaliforniaHousing/cal_housing.data", names=columns, header=None)
cal_housing_df["medianHouseValue"] /= 500000 # Scaling
cal_housing_df.to_csv(f"./data/raw/raw_data_all.csv", header=True, index=False)
rawdata_s3_prefix = "{}/data/raw".format(prefix)
raw_s3 = sagemaker_session.upload_data(path="./data/raw/", key_prefix=rawdata_s3_prefix)
print(raw_s3)

s3://sagemaker-ap-northeast-2-532805286864/pipeline-model-example/data/raw


### Define Parameters to Parameterize Pipeline Execution

In [9]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat

# raw input data
input_data = ParameterString(name="inputData", default_value=raw_s3)

# status of newly trained model in registry
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="Approved")

# processing step parameters
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

# training step parameters
training_instance_type = ParameterString("TrainingInstanceType", default_value="ml.m5.xlarge")

# 실제 수행 시 epoch 수정할 것
training_epochs = ParameterString("TrainingEpochs", default_value="1")

# model performance step parameters
# 초기 기능 테스틀 위해 0.75 -> 0.51로 변경함.
accuracy_mse_threshold = ParameterFloat(name="AccuracyMseThreshold", default_value=0.51)

### Define a Processing Step for Feature Engineering

The below preprocessing script, in addition to creating a scaler, contains the necessary functions for it to be deployed as part of a pipeline model

In [10]:
!mkdir -p code

In [11]:
%%writefile code/preprocess.py

import glob
import numpy as np
import pandas as pd
import os
import json
import joblib
from io import StringIO
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tarfile

try:
    from sagemaker_containers.beta.framework import (
        content_types,
        encoders,
        env,
        modules,
        transformer,
        worker,
        server,
    )
except ImportError:
    pass

feature_columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
]
label_column = "medianHouseValue"

base_dir = "/opt/ml/processing"
base_output_dir = "/opt/ml/output/"

if __name__ == "__main__":
    df = pd.read_csv(f"{base_dir}/input/raw_data_all.csv")
    feature_data = df.drop(label_column, axis=1, inplace=False)
    label_data = df[label_column]
    x_train, x_test, y_train, y_test = train_test_split(feature_data, label_data, test_size=0.33)

    scaler = StandardScaler()

    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    train_dataset = pd.concat([pd.DataFrame(x_train), y_train.reset_index(drop=True)], axis=1)
    test_dataset = pd.concat([pd.DataFrame(x_test), y_test.reset_index(drop=True)], axis=1)

    train_dataset.columns = feature_columns + [label_column]
    test_dataset.columns = feature_columns + [label_column]

    train_dataset.to_csv(f"{base_dir}/train/train.csv", header=True, index=False)
    test_dataset.to_csv(f"{base_dir}/test/test.csv", header=True, index=False)
    joblib.dump(scaler, "model.joblib")
    with tarfile.open(f"{base_dir}/scaler_model/model.tar.gz", "w:gz") as tar_handle:
        tar_handle.add(f"model.joblib")


def input_fn(input_data, content_type):
    """Parse input data payload

    We currently only take csv input. Since we need to process both labelled
    and unlabelled data we first determine whether the label column is present
    by looking at how many columns were provided.
    """
    if content_type == "text/csv":
        # Read the raw input data as CSV.
        df = pd.read_csv(StringIO(input_data), header=None)

        if len(df.columns) == len(feature_columns) + 1:
            # This is a labelled example, includes the ring label
            df.columns = feature_columns + [label_column]
        elif len(df.columns) == len(feature_columns):
            # This is an unlabelled example.
            df.columns = feature_columns

        return df
    else:
        raise ValueError("{} not supported by script!".format(content_type))


def output_fn(prediction, accept):
    """Format prediction output

    The default accept/content-type between containers for serial inference is JSON.
    We also want to set the ContentType or mimetype as the same value as accept so the next
    container can read the response payload correctly.
    """
    if accept == "application/json":
        instances = []
        for row in prediction.tolist():
            instances.append(row)
        json_output = {"instances": instances}

        return worker.Response(json.dumps(json_output), mimetype=accept)
    elif accept == "text/csv":
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException("{} accept type is not supported by this script.".format(accept))


def predict_fn(input_data, model):
    """Preprocess input data

    We implement this because the default predict_fn uses .predict(), but our model is a preprocessor
    so we want to use .transform().

    The output is returned in the following order:

        rest of features either one hot encoded or standardized
    """
    features = model.transform(input_data)

    if label_column in input_data:
        # Return the label (as the first column) and the set of features.
        return np.insert(features, 0, input_data[label_column], axis=1)
    else:
        # Return only the set of features
        return features


def model_fn(model_dir):
    """Deserialize fitted model"""
    preprocessor = joblib.load(os.path.join(model_dir, "model.joblib"))
    return preprocessor

Overwriting code/preprocess.py


### Preprocessing 설정

In [12]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version = sklearn_framework_version,
    instance_type = "ml.m5.large",
    instance_count = processing_instance_count,
    # base_job_name = "sklearn-housing-data-process",   # just prefix
    role=role,
    sagemaker_session=pipeline_session,
)

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="scaler_model", source="/opt/ml/processing/scaler_model"),
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocess.py",
    job_name="housing-data-process",
)
    

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [13]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

step_process = ProcessingStep(
    name="PreprocessData",
    step_args=processor_args,
)

### Define a Training Step to Train Model

In [14]:
%%writefile code/train.py

import argparse
import numpy as np
import os
import tensorflow as tf
import pandas as pd

feature_columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
]
label_column = "medianHouseValue"

def parse_args():
    parser = argparse.ArgumentParser()
    
    # hyperparameters sent by the client are passed as command-line arguements to to script
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--learning_rate", type=float, default=0.1)
    
    # data directories
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    
    # model directory
    parser.add_argument("--sm-model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    
    return parser.parse_known_args()

def get_train_data(train_dir):
    train_data = pd.read_csv(os.path.join(train_dir, "train.csv"))
    x_train = train_data[feature_columns].to_numpy()
    y_train = train_data[label_column].to_numpy()
    print("x train: ", x_train.shape, ", y train: ", y_train.shape)
    
    return x_train, y_train

def get_test_data(test_dir):
    test_data = pd.read_csv(os.path.join(test_dir, "test.csv"))
    x_test = test_data[feature_columns].to_numpy()
    y_test = test_data[label_column].to_numpy()
    
    return x_test, y_test

def get_model():
    inputs = tf.keras.Input(shape=(8,))
    hidden_1 = tf.keras.layers.Dense(8, activation="tanh")(inputs)
    hidden_2 = tf.keras.layers.Dense(4, activation="sigmoid")(hidden_1)
    outputs = tf.keras.layers.Dense(1)(hidden_2)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

if __name__ == "__main__":
    args, _ = parse_args()
    
    print("Training data location: {}".format(args.train))
    print("Test data location: {}".format(args.test))
    
    # 서버에서 모델 위치 확인 용
    print("model location is {}".format(args.sm_model_dir))
    
    x_train, y_train = get_train_data(args.train)
    x_test, y_test = get_test_data(args.test)
    
    
    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate
    print(
        "batch_size = {}, epochs = {}, learning_rate = {}".format(batch_size, epochs, learning_rate)
    )
    
    model = get_model()
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(optimizer=optimizer, loss="mse")
    model.fit(
        x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test)
    )
    
    # evaluate on test set
    scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
    print("\nTest MSE: ", scores)
    
    # save model
    model.save(args.sm_model_dir + "/1")
    

Overwriting code/train.py


In [15]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.model_step import ModelStep
import time

# Where to store the trained model
model_path = f"s3://{bucket}/{prefix}/model/"

hyperparameters = {"epochs": training_epochs }
tensorflow_version = "2.4.1"
python_version = "py37"

tf2_estimator = TensorFlow(
    source_dir="code",
    entry_point="train.py",
    instance_type=training_instance_type,
    instance_count=1,
    framework_version=tensorflow_version,
    role=role,
    # base_job_name="tensorflow-train-model",
    output_path=model_path,
    hyperparameters=hyperparameters,
    py_version=python_version,
    sagemaker_session=pipeline_session,
)

# Note how the input to the training job directly references the output of the previous step
train_args = tf2_estimator.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "test": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

step_train_model = TrainingStep(name="TrainTensorflowModel", step_args=train_args)

### Define a Model Evaluation Step to Evaluate the Trained Model

In [16]:
%%writefile code/evaluate.py

import os
import json
import sys
import numpy as np
import pandas as pd
import pathlib
import tarfile

feature_columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
]
label_column = "medianHouseValue"

if __name__ == "__main__":
    # train.py에서 model seve한 location
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path, "r:gz") as tar:
        tar.extractall("./model")
    import tensorflow as tf
    
    model = tf.keras.models.load_model("./model/1")
    test_path = "/opt/ml/processing/test/"
    df = pd.read_csv(test_path + "/test.csv")
    x_test = df[feature_columns].to_numpy()
    y_test = df[label_column].to_numpy()
    scores = model.evaluate(x_test, y_test, verbose=2)
    print("\nTest MSE: ", scores)
    
    # Available metrics to add to model: https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html
    report_dict = {
        "regression_metrics": {
            "mse": {"value": scores, "standard_deviation": "NaN"},
        },
    }
    
    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))


Overwriting code/evaluate.py


In [17]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.sklearn.processing import  ScriptProcessor

# tensorflow image 및 instance 지정
tf_eval_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version=tensorflow_version,
    image_scope="training",
    py_version="py37",
    instance_type="ml.m5.xlarge",
)

# Processor 지정
evaluate_model_processor = ScriptProcessor(
    role=role,
    image_uri=tf_eval_image_uri,
    command=["python3"],
    instance_count=1,
    instance_type=processing_instance_type,
    sagemaker_session=pipeline_session,
)
# Create a PropertFile
# A PropertyFile is used to be able to reference outputs from a processing step, for instance to use in a condition step.
# For more information, visit https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html
evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

eval_args = evaluate_model_processor.run(
    # evaluate.py의 입력 arg로 들어감.
    inputs=[
        ProcessingInput(
            source=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    # PropertyFile의 output_name과 동일해야 함.
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code = "code/evaluate.py",
    job_name="evaluation_job"
)

step_evaluate_model = ProcessingStep(
    name="EvaluateModelPerformance",
    step_args=eval_args,
    property_files=[evaluation_report],
)
    
    

### Define a Register Model Step to Create a Model Package for the PipelineModel
Processing Job name이 Unique해야 되고, 이를 이용하는 Job은 그 Job과 동시에 돌려야 함. 검증 차원에서 별도로 돌리기 어렵다.

In [18]:
from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import PipelineModel

# step_process로부터 정보를 가져 옴.
scaler_model_s3 = "{}/model.tar.gz".format(
    step_process.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
)

scaler_model = SKLearnModel(
    model_data=scaler_model_s3,
    role=role,
    sagemaker_session=pipeline_session,
    entry_point="code/preprocess.py",
    framework_version=sklearn_framework_version,
)

tf_model_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version=tensorflow_version,
    image_scope="inference",
    py_version="py37",
    instance_type="ml.m5.xlarge",
)

tf_model = Model(
    image_uri=tf_model_image_uri,
    model_data=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

pipeline_model = PipelineModel(
    models=[scaler_model, tf_model], role=role, sagemaker_session=pipeline_session
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py37.


In [19]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

evaluation_s3_uri = "{}/evaluation.json".format(
    step_evaluate_model.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

register_args = pipeline_model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
    approval_status=model_approval_status,
)

step_register_pipeline_model = ModelStep(
    name="PipelineModel",
    step_args=register_args,
)

### Define a Condition Step to Check Accuracy and Conditionally Register a Model in the Model Registry

In [20]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# Create accuracy condtion to ensure the model meet performance requirements.
# Models with a test accuracy lower than the condition will not be registered with the model registry.
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_evaluate_model.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=accuracy_mse_threshold,
)

# Create a Sagemaker Pipelines ConditionStep, using the condition above.
# Enter the steps to perform if the condition returns True / False.
step_cond = ConditionStep(
    name="MSE-Lower-Than-Threshold-Condition",
    conditions=[cond_lte],
    if_steps=[step_register_pipeline_model],
    else_steps=[],
)

### Define a Pipeline of Parameters, Steps, and Conditions

In [21]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

# Create a definition configuration and toggle on custom prefixing
definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True);

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        training_epochs,
        input_data,
        model_approval_status,
        accuracy_mse_threshold,
    ],
    steps=[step_process, step_train_model, step_evaluate_model, step_cond],
    pipeline_definition_config=definition_config,
)

import json

definition = json.loads(pipeline.definition())
print(definition)



INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


{'Version': '2020-12-01', 'Metadata': {}, 'Parameters': [{'Name': 'ProcessingInstanceType', 'Type': 'String', 'DefaultValue': 'ml.m5.xlarge'}, {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1}, {'Name': 'TrainingInstanceType', 'Type': 'String', 'DefaultValue': 'ml.m5.xlarge'}, {'Name': 'TrainingEpochs', 'Type': 'String', 'DefaultValue': '1'}, {'Name': 'inputData', 'Type': 'String', 'DefaultValue': 's3://sagemaker-ap-northeast-2-532805286864/pipeline-model-example/data/raw'}, {'Name': 'ModelApprovalStatus', 'Type': 'String', 'DefaultValue': 'Approved'}, {'Name': 'AccuracyMseThreshold', 'Type': 'Float', 'DefaultValue': 0.51}], 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'}, 'TrialName': {'Get': 'Execution.PipelineExecutionId'}}, 'Steps': [{'Name': 'PreprocessData', 'Type': 'Processing', 'Arguments': {'ProcessingJobName': 'housing-data-process', 'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.large', 'InstanceCount

### Submit the pipeline to SageMaker and start execution

In [22]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


### Deploy latest approved model to a real-time endpoint

In [23]:
%%writefile utils.py
import argparse
import boto3
import logging
import os
from botocore.exceptions import ClientError
import tarfile
import zipfile

logger = logging.getLogger(__name__)
sm_client = boto3.client("sagemaker")


def get_approved_package(model_package_group_name):
    """Gets the latest approved model package for a model package group.

    Args:
        model_package_group_name: The model package group name.

    Returns:
        The SageMaker Model Package ARN.
    """
    try:
        # Get the latest approved model package
        response = sm_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # Fetch more packages if none returned with continuation token
        while len(approved_packages) == 0 and "NextToken" in response:
            logger.debug("Getting more packages for token: {}".format(response["NextToken"]))
            response = sm_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        # Return error if no packages found
        if len(approved_packages) == 0:
            error_message = (
                f"No approved ModelPackage found for ModelPackageGroup: {model_package_group_name}"
            )
            logger.error(error_message)
            raise Exception(error_message)

        # Return the pmodel package arn
        model_package_arn = approved_packages[0]["ModelPackageArn"]
        logger.info(f"Identified the latest approved model package: {model_package_arn}")
        return approved_packages[0]
        # return model_package_arn
    except ClientError as e:
        error_message = e.response["Error"]["Message"]
        logger.error(error_message)
        raise Exception(error_message)

Overwriting utils.py


In [24]:
import boto3
from utils import get_approved_package

sm_client = boto3.client("sagemaker")

pck = get_approved_package(model_package_group_name)
print(pck["ModelPackageArn"])

model_description = sm_client.describe_model_package(ModelPackageName=pck["ModelPackageArn"])
model_description

INFO:utils:Identified the latest approved model package: arn:aws:sagemaker:ap-northeast-2:532805286864:model-package/PipelineModelPackageGroup/2


arn:aws:sagemaker:ap-northeast-2:532805286864:model-package/PipelineModelPackageGroup/2


{'ModelPackageGroupName': 'PipelineModelPackageGroup',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:532805286864:model-package/PipelineModelPackageGroup/2',
 'CreationTime': datetime.datetime(2024, 4, 5, 2, 8, 14, 90000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3',
    'ImageDigest': 'sha256:93e5fa7425580265fc8e0b1c06f51ff7a2690adf844cd9b8281634b251c593c1',
    'ModelDataUrl': 's3://sagemaker-ap-northeast-2-532805286864/housing-data-process/output/scaler_model/model.tar.gz',
    'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
     'SAGEMAKER_PROGRAM': 'preprocess.py',
     'SAGEMAKER_REGION': 'ap-northeast-2',
     'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-ap-northeast-2-532805286864/sagemaker-scikit-learn-2024-04-05-01-56-26-619/sourcedir.tar.gz'}},
   {'Image': '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-

In [25]:
from sagemaker import ModelPackage

model_package_arn = model_description["ModelPackageArn"]
model = ModelPackage(role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session)
endpoint_name = "DEMO-endpoint-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("EndpointName= {}".format(endpoint_name))
model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=endpoint_name)

EndpointName= DEMO-endpoint-2024-04-05-02-08-31


INFO:sagemaker:Creating model with name: PipelineModelPackageGroup-2024-04-05-02-08-31-975
INFO:sagemaker:Creating endpoint-config with name DEMO-endpoint-2024-04-05-02-08-31
INFO:sagemaker:Creating endpoint with name DEMO-endpoint-2024-04-05-02-08-31


------!

In [26]:
from sagemaker.predictor import Predictor
predictor = Predictor(endpoint_name=endpoint_name)

In [27]:
data = pd.read_csv("data/raw/raw_data_all.csv")
house_values = data["medianHouseValue"]
data  = data.drop("medianHouseValue", axis=1)

pred_count = 10
payload = data.iloc[:pred_count].to_csv(header=False, index=False)
p = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
print(p.decode("utf-8"))

{
    "predictions": [[0.650978863], [0.566298366], [0.649416566], [0.556757569], [0.422180712], [0.398475975], [0.506694853], [0.511842191], [0.339357406], [0.559426785]
    ]
}


In [28]:
blue, stop = "\033[94m", "\033[0m"
predictions = json.loads(p.decode("utf-8"))["predictions"]
for i in range(pred_count):
    print(
        f"Predicted: {blue}{predictions[i][0]}{stop} and Actual is: {blue}{house_values.iloc[i]}{stop}"
    )

Predicted: 0.650978863 and Actual is: 0.9052
Predicted: 0.566298366 and Actual is: 0.717
Predicted: 0.649416566 and Actual is: 0.7042
Predicted: 0.556757569 and Actual is: 0.6826
Predicted: 0.422180712 and Actual is: 0.6844
Predicted: 0.398475975 and Actual is: 0.5394
Predicted: 0.506694853 and Actual is: 0.5984
Predicted: 0.511842191 and Actual is: 0.4828
Predicted: 0.339357406 and Actual is: 0.4534
Predicted: 0.559426785 and Actual is: 0.5222


### Clean-Up

In [34]:
sm_client = boto3.client("sagemaker")

for d in sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]:
    print(d["ModelPackageArn"])
    sm_client.delete_model_package(ModelPackageName=d["ModelPackageArn"])

sm_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

arn:aws:sagemaker:ap-northeast-2:532805286864:model-package/PipelineModelPackageGroup/2
arn:aws:sagemaker:ap-northeast-2:532805286864:model-package/PipelineModelPackageGroup/1


{'ResponseMetadata': {'RequestId': '84708a1e-664a-4be7-8310-32a539efcd79',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '84708a1e-664a-4be7-8310-32a539efcd79',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 05 Apr 2024 02:25:02 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [35]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: DEMO-endpoint-2024-04-05-02-08-31
INFO:sagemaker:Deleting endpoint with name: DEMO-endpoint-2024-04-05-02-08-31


In [36]:
pipeline.delete()

INFO:sagemaker.workflow.pipeline:If triggers have been setup for this target, they will become orphaned.You will need to clean them up manually via the CLI or EventBridge console.


{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:532805286864:pipeline/serial-inference-pipeline',
 'ResponseMetadata': {'RequestId': 'b2856691-41f8-48f4-8e0f-3b23e7f61916',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b2856691-41f8-48f4-8e0f-3b23e7f61916',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Fri, 05 Apr 2024 02:25:37 GMT'},
  'RetryAttempts': 0}}